Importing libraries

In [1]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\Additional-Task\\storage_key.json"

In [5]:
d = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
d

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...
...,...,...
381,PANT,https://m.media-amazon.com/images/I/51FJIrVNyd...
382,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...
383,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...
384,TV,https://static1.pocketlintimages.com/wordpress...


Reading csv file containing product_name and url

In [6]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
data.shape

(386, 2)

Storing url column in data_url

In [17]:
data_url = data['product_url']
data_url

0      https://m.media-amazon.com/images/I/71xWtLhH2G...
1      https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2      https://m.media-amazon.com/images/I/71YG7EClYf...
3      https://m.media-amazon.com/images/I/81ATe15IyH...
4      https://m.media-amazon.com/images/I/61OBcY37KX...
                             ...                        
381    https://m.media-amazon.com/images/I/51FJIrVNyd...
382    https://m.media-amazon.com/images/I/81mVIqIYZu...
383    https://m.media-amazon.com/images/I/61l9ppRIiq...
384    https://static1.pocketlintimages.com/wordpress...
385    https://www.jansatta.com/wp-content/uploads/20...
Name: product_url, Length: 386, dtype: object

Function to check if url contains a image or not

In [4]:
import requests
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers["content-type"] in image_formats:
      # print(r.headers["content-type"])
      return True
   return False

In [5]:
is_url_image('https://m.media-amazon.com/images/I/71vdb0uL3xL._SX695_.jpg')

False

In [8]:
import webcolors

def closest_colour(requested_colour):
    r_req, g_req, b_req = requested_colour
    min_diff = float('inf')
    closest_name = None
    
    for hex_value, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(hex_value)
        diff = (r_c - r_req) ** 2 + (g_c - g_req) ** 2 + (b_c - b_req) ** 2
        if diff < min_diff:
            min_diff = diff
            closest_name = name
    
    return closest_name

def get_colour_name(requested_colour):
    try:
        return webcolors.rgb_to_name(requested_colour)
    except ValueError:
        return closest_colour(requested_colour)


In [12]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    dominant_color = props.dominant_colors.colors[0].color
    rgb = (dominant_color.red, dominant_color.green, dominant_color.blue)
    return get_colour_name(rgb)
    

In [13]:
detect_properties_uri("https://m.media-amazon.com/images/I/612I0tsJwQS._AC_UL320_.jpg")

'lightsteelblue'

In [20]:
colors=[]
for i in data_url:
    co = detect_properties_uri(i)
    colors.append(co)
    

In [21]:
colors

['darkslategray',
 'rosybrown',
 'darkslategray',
 'dimgray',
 'lavender',
 'black',
 'darkslategray',
 'black',
 'white',
 'maroon',
 'black',
 'darkslateblue',
 'gainsboro',
 'ghostwhite',
 'white',
 'darkslateblue',
 'gray',
 'linen',
 'dimgray',
 'darkslategray',
 'silver',
 'gainsboro',
 'black',
 'gainsboro',
 'white',
 'steelblue',
 'mistyrose',
 'gainsboro',
 'darkslategray',
 'darkgray',
 'darkgray',
 'darkgray',
 'silver',
 'gainsboro',
 'khaki',
 'cadetblue',
 'tomato',
 'salmon',
 'lightcoral',
 'rosybrown',
 'darkcyan',
 'darkslategray',
 'midnightblue',
 'teal',
 'black',
 'darkslategray',
 'slateblue',
 'dimgray',
 'darkorchid',
 'rosybrown',
 'sandybrown',
 'darkmagenta',
 'tomato',
 'tomato',
 'darkorange',
 'chocolate',
 'olivedrab',
 'silver',
 'cornflowerblue',
 'paleturquoise',
 'cadetblue',
 'palevioletred',
 'lavender',
 'gray',
 'darkslategray',
 'dimgray',
 'darkslategray',
 'darkslategray',
 'darkslategray',
 'black',
 'snow',
 'darkslategray',
 'black',
 'dim

In [25]:
df['color'] = colors

In [26]:
df

,product_name,product_url,object,label,color
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'...",darkslategray
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ...",rosybrown
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ...",darkslategray
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ...",dimgray
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '...",lavender
...,...,...,...,...,...
381,PANT,https://m.media-amazon.com/images/I/51FJIrVNyd...,['Pants'],"['Shoe', 'Active pants', 'Sleeve', 'Gesture', ...",gray
382,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...,{'Pants'},"['Trousers', 'Leg', 'Blue', 'Active pants', 'H...",black
383,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...,['Mobile phone'],"['Product', 'Rectangle', 'Purple', 'Violet', '...",hotpink
384,TV,https://static1.pocketlintimages.com/wordpress...,{'Television'},"['Flat panel display', 'Font', 'Television set...",burlywood


In [12]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[1])

gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined


Empty String variable to store results

In [30]:
txt = ""

Function to extract information from image

In [9]:
def extract_info(uri):
    # global txt
    # try:
        # if(is_url_image(uri)==True):
            txt = ''
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri

            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]

            response = client.image_properties(image=img)
            props = response.image_properties_annotation
            dominant_color = props.dominant_colors.colors[0].color
            rgb = (dominant_color.red, dominant_color.green, dominant_color.blue)

            label.append(get_colour_name(rgb))
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            
            txt = list(set(obj).intersection(label)) if set(obj).intersection(label) else label[:2]
            
            return txt, label
    #     else:
    #         txt = "Please enter a valid URL that contains Image"
    # except Exception as e:
    #     txt = f"Please Enter a Valid URL of Image"
    # return str(txt)

In [40]:
du = data_url[:10]
du

0    https://m.media-amazon.com/images/I/71xWtLhH2G...
1    https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2    https://m.media-amazon.com/images/I/71YG7EClYf...
3    https://m.media-amazon.com/images/I/81ATe15IyH...
4    https://m.media-amazon.com/images/I/61OBcY37KX...
5    https://m.media-amazon.com/images/I/710+f7XX2F...
6    https://m.media-amazon.com/images/I/71u2XOFXAI...
7    https://m.media-amazon.com/images/I/61U0BBQYhr...
8    https://m.media-amazon.com/images/I/71IQc8cAIQ...
9    https://m.media-amazon.com/images/I/71fKZrHSO9...
Name: product_url, dtype: object

In [41]:
d = data[:10]
d

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...
5,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...
6,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...
7,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...
8,ADIDAS Mens Regular Fit M Sereno Bos T2 Jerseys,https://m.media-amazon.com/images/I/71IQc8cAIQ...
9,adidas Men's Regular Fit M D2M Polo,https://m.media-amazon.com/images/I/71fKZrHSO9...


Creating new dataframe to save results

In [42]:
result = data
result.head()

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...


Iterating through function to extract features of image and adding it to result dataframe

In [43]:
for i in range(len(du)):
    print(i)
    extract_info(du[i])
    result.loc[i, "objects_extracted"] = txt
    

0


C:\Users\Microsoft\AppData\Local\Temp\ipykernel_11244\4098288774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.loc[i, "objects_extracted"] = txt


1
2
3
4


ValueError: Must have equal len keys and value when setting with an iterable

Resultant Dataframe

In [35]:
result

,product_name,product_url,objects_extracted
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,None
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,None
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,None
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,None
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,None
5,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...,None
6,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...,None
7,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...,None
8,ADIDAS Mens Regular Fit M Sereno Bos T2 Jerseys,https://m.media-amazon.com/images/I/71IQc8cAIQ...,None
9,adidas Men's Regular Fit M D2M Polo,https://m.media-amazon.com/images/I/71fKZrHSO9...,None


Saving Dataframe to CSV file

In [4]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

NameError: name 'result' is not defined

In [5]:
nnn = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
nnn.shape

(0, 3)

In [14]:
nnn.to_csv("gs://bucket-shreyash/Product_Data/Product_D.csv",index=False)

In [22]:
df= pd.read_csv("gs://bucket-shreyash/Product_Data/Product_D.csv")
df

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
381,PANT,https://m.media-amazon.com/images/I/51FJIrVNyd...,['Pants'],"['Shoe', 'Active pants', 'Sleeve', 'Gesture', ..."
382,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...,{'Pants'},"['Trousers', 'Leg', 'Blue', 'Active pants', 'H..."
383,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...,['Mobile phone'],"['Product', 'Rectangle', 'Purple', 'Violet', '..."
384,TV,https://static1.pocketlintimages.com/wordpress...,{'Television'},"['Flat panel display', 'Font', 'Television set..."


In [27]:
df.to_csv('D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv', index=False)

In [21]:
print(df.to_string())

                                                                                                        product_name                                                                                                                                                                         product_url                                                                                                                                                         object                                                                                                                                                                                     label
0                                                                                       Sparx Men's Sc0162g Sneakers                                                                                                                         https://m.media-amazon.com/images/I/71xWtLhH2GL._SX695_.jpg                                                              

In [31]:
v_df = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")

v_df

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
381,PANT,https://m.media-amazon.com/images/I/51FJIrVNyd...,['Pants'],"['Shoe', 'Active pants', 'Sleeve', 'Gesture', ..."
382,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...,{'Pants'},"['Trousers', 'Leg', 'Blue', 'Active pants', 'H..."
383,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...,['Mobile phone'],"['Product', 'Rectangle', 'Purple', 'Violet', '..."
384,TV,https://static1.pocketlintimages.com/wordpress...,{'Television'},"['Flat panel display', 'Font', 'Television set..."


str

In [7]:
finalDF = pd.concat([df, v_df], ignore_index=True)
finalDF

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
622,TV,https://www.jansatta.com/wp-content/uploads/20...,NaN,NaN
623,TOP,https://i.pinimg.com/originals/2e/88/a5/2e88a5...,NaN,NaN
624,WATCH,https://w7.pngwing.com/pngs/566/206/png-transp...,NaN,NaN
625,SHIRT,https://funkybabaji.com/cdn/shop/files/9.png?v...,NaN,NaN


In [9]:
finalDF['product_url'].duplicated().sum()

237

In [10]:
hehehe = finalDF.drop_duplicates(subset=['product_url'])

In [32]:
v_df

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
381,PANT,https://m.media-amazon.com/images/I/51FJIrVNyd...,['Pants'],"['Shoe', 'Active pants', 'Sleeve', 'Gesture', ..."
382,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...,{'Pants'},"['Trousers', 'Leg', 'Blue', 'Active pants', 'H..."
383,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...,['Mobile phone'],"['Product', 'Rectangle', 'Purple', 'Violet', '..."
384,TV,https://static1.pocketlintimages.com/wordpress...,{'Television'},"['Flat panel display', 'Font', 'Television set..."


In [33]:
v_df.to_csv("gs://bucket-shreyash/Product_Data/Product_DC.csv",index=False)

In [11]:
print(hehehe.to_string())

                                                                                                        product_name                                                                                                                                                                         product_url                                                                                                                                                         object                                                                                                                                                                                 label
0                                                                                       Sparx Men's Sc0162g Sneakers                                                                                                                         https://m.media-amazon.com/images/I/71xWtLhH2GL._SX695_.jpg                                                                  

In [18]:
nnnew = hehehe.iloc[288:]
nnnew = nnnew[['product_name','product_url']]
nnnew

,product_name,product_url
288,BOTTLE,https://m.media-amazon.com/images/I/51Np+iuzhS...
289,BOTTLE,https://m.media-amazon.com/images/I/51Da+TBexC...
290,BOTTLE,https://m.media-amazon.com/images/I/41GC5Z0Frh...
291,BOTTLE,https://m.media-amazon.com/images/I/61c-GtJ+0e...
292,BOTTLE,https://m.media-amazon.com/images/I/611Ugi6GpN...
...,...,...
618,PANT,https://m.media-amazon.com/images/I/81mVIqIYZu...
619,PHONE,https://m.media-amazon.com/images/I/61l9ppRIiq...
620,TV,https://static1.pocketlintimages.com/wordpress...
621,TV,https://yuppstatic.akamaized.net/yupptv/images...


In [19]:
nnnew.to_csv("D:\\Work and Assignments\\Python\\sampleCSV.csv",index=False)

In [12]:
query='shoe'

In [16]:
if df['product_name'].str.lower().str.contains(query).any():
    name = df[df['product_name'].str.lower().str.contains(query)]['product_name'].tolist()
    print(name)
    link = df[df['product_name'].str.lower().str.contains(query)]['product_url'].tolist()

['Puma Womens Kendall WNS Running Shoe', "Nivia Men's Combat 2.0 Basketball Shoes", 'adidas Mens Amalgo M Running Shoe', 'Red Tape Casual Sneaker Shoes for Men', 'Bata Mens MaxUniform Dress Shoe', 'SPARX Mens Sx0675g Walking Shoe']


In [9]:

namee = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes').any()]['product_name'].tolist()
link = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes')]['product_url'].tolist()

In [10]:
namee

['Leader Spyder 27.5T MTB Cycle/Bike',
 'Leader Scout MTB 26T Mountain Bicycle/Bike',
 'Amitasha Pull Back Motorbike High-Speed Racing Bike',
 'Baybee R7 Battery Operated Bike for Kids',
 'Nexus Product Bike for Kids Toy R3 Bike',
 'PLUSPOINT Diecast Motorcycle Toy Ducati Bike']

In [25]:
d = a['product_name'].map(lambda x: x.lower())

In [26]:
d

0                           sparx men's sc0162g sneakers
1                   puma womens kendall wns running shoe
2                nivia men's combat 2.0 basketball shoes
3                      adidas mens amalgo m running shoe
4                  red tape casual sneaker shoes for men
                             ...                        
152    gormery 1/12 scale model compatible with kawasaki
153              nexus product bike for kids toy r3 bike
154         pluspoint diecast motorcycle toy ducati bike
155                 solakaka sm600 wireless gaming mouse
156    zebronics zeb-blaze rgb gaming mouse pad with ...
Name: product_name, Length: 157, dtype: object

In [78]:
ob = nnn['objects_extracted']

In [91]:
import ast

e = {'Television'}
for i in ob:
    i = set(ast.literal_eval(i))
    if len(e.intersection(i)) !=0:
        print(i)


{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}


Uploading csv file to Cloud Stroage

In [2]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed


In [46]:
import numpy as np

def cosine_similarity_rgb(color1, color2):
    # Convert RGB tuples to numpy arrays
    vec1 = np.array(color1)
    vec2 = np.array(color2)
    
    # Compute the dot product
    dot_product = np.dot(vec1, vec2)
    
    # Compute the magnitudes of the vectors
    magnitude1 = np.linalg.norm(vec1)
    magnitude2 = np.linalg.norm(vec2)
    
    # Compute the cosine similarity
    if magnitude1 == 0 or magnitude2 == 0:
        return 0  # To avoid division by zero, though RGB vectors should not be zero
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    
    return cosine_similarity

# Example usage
color1 = (0, 255, 0)  # Red
color2 = (255, 255, 255)  # Green

similarity_score = cosine_similarity_rgb(color1, color2)
print(f"The cosine similarity score between the colors is: {similarity_score}")


The cosine similarity score between the colors is: 0.5773502691896257
